In [15]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [16]:
# Import our input dataset
attrition_df = pd.read_csv('./Resources/HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [17]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [18]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [19]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [20]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

C:\Users\berge\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [21]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\berge\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 448       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 1ms/step - loss: 960.1417 - accuracy: 0.8276
Epoch 2/100
35/35 [==============================] - 0s 1ms/step - loss: 569.1102 - accuracy: 0.8276
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 279.7077 - accuracy: 0.8276
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 116.1201 - accuracy: 0.8149
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 26.1195 - accuracy: 0.5971
Epoch 6/100
35/35 [==============================] - 0s 1ms/step - loss: 9.8110 - accuracy: 0.3857
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 5.7740 - accuracy: 0.3212
Epoch 8/100
35/35 [==============================] - 0s 1ms/step - loss: 4.2953 - accuracy: 0.2931
Epoch 9/100
35/35 [==============================] - 0s 1ms/step - loss: 3.2032 - accuracy: 0.7205
Epoch 10/100
35/35 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.8276


35/35 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.8294
Epoch 83/100
35/35 [==============================] - 0s 988us/step - loss: 0.4574 - accuracy: 0.8294
Epoch 84/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4572 - accuracy: 0.8294
Epoch 85/100
35/35 [==============================] - 0s 922us/step - loss: 0.4571 - accuracy: 0.8294
Epoch 86/100
35/35 [==============================] - 0s 903us/step - loss: 0.4570 - accuracy: 0.8294
Epoch 87/100
35/35 [==============================] - 0s 912us/step - loss: 0.4569 - accuracy: 0.8294
Epoch 88/100
35/35 [==============================] - 0s 981us/step - loss: 0.4569 - accuracy: 0.8294
Epoch 89/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4568 - accuracy: 0.8294
Epoch 90/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4567 - accuracy: 0.8294
Epoch 91/100
35/35 [==============================] - 0s 1000us/step - loss: 0.4566 - accuracy: 0.8294


In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3924 - accuracy: 0.8696 - 100ms/epoch - 8ms/step
Loss: 0.3924371600151062, Accuracy: 0.8695651888847351


In [27]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 1/35 [..............................] - ETA: 10s - loss: 0.6221 - accuracy: 0.8438
Epoch 1: saving model to checkpoints\weights.01.hdf5
35/35 [==============================] - 1s 13ms/step - loss: 0.6554 - accuracy: 0.8276
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.3382 - accuracy: 0.9375
Epoch 2: saving model to checkpoints\weights.02.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.5622 - accuracy: 0.8276
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.5115 - accuracy: 0.8438
Epoch 3: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.5088 - accuracy: 0.8276
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.4524 - accuracy: 0.8438
Epoch 4: saving model to checkpoints\weights.04.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.4795 - accuracy: 0.8276
Epoch 5/100
 1/35 [..............................] - ETA: 

Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.2506 - accuracy: 0.8750
Epoch 36: saving model to checkpoints\weights.36.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3228 - accuracy: 0.8276
Epoch 37/100
 1/35 [..............................] - ETA: 0s - loss: 0.2616 - accuracy: 0.9062
Epoch 37: saving model to checkpoints\weights.37.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3191 - accuracy: 0.8276
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.3464 - accuracy: 0.8125
Epoch 38: saving model to checkpoints\weights.38.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3147 - accuracy: 0.8276
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.4921 - accuracy: 0.7188
Epoch 39: saving model to checkpoints\weights.39.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3119 - accuracy: 0.8276
Epoch 40/100
 1/35 [..............................] 

35/35 [==============================] - 0s 1ms/step - loss: 0.2150 - accuracy: 0.8276
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.2683 - accuracy: 0.7500
Epoch 71: saving model to checkpoints\weights.71.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2127 - accuracy: 0.8276
Epoch 72/100
 1/35 [..............................] - ETA: 0s - loss: 0.2070 - accuracy: 0.8750
Epoch 72: saving model to checkpoints\weights.72.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2099 - accuracy: 0.8276
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.2743 - accuracy: 0.8438
Epoch 73: saving model to checkpoints\weights.73.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2082 - accuracy: 0.8276
Epoch 74/100
 1/35 [..............................] - ETA: 0s - loss: 0.1748 - accuracy: 0.8125
Epoch 74: saving model to checkpoints\weights.74.hdf5
35/35 [==============================] - 0s 1ms/step

In [29]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.5989 - accuracy: 0.8315 - 160ms/epoch - 13ms/step
Loss: 0.5989086627960205, Accuracy: 0.83152174949646


In [30]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [31]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [32]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.5989 - accuracy: 0.8315 - 99ms/epoch - 8ms/step
Loss: 0.5989086627960205, Accuracy: 0.83152174949646
